In [1]:
!pip install tensorflow==1.13.1
!pip install FFmpeg
!pip install ffmpeg-python

In [1]:
import os
import time
import cv2
import numpy as np
import PIL.Image as Image
import random
import tensorflow as tf
import threading
from datetime import datetime
import csv

In [2]:
def _variable_on_cpu(name, shape, initializer, trainable):
    with tf.device('/cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer, trainable=trainable)
    return var

def _variable_with_weight_decay(name, shape, wd, trainable):
    var = _variable_on_cpu(name, shape, tf.contrib.layers.xavier_initializer(), trainable=trainable)
    #Check this weight_decay loss!
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_decay_loss')
        tf.add_to_collection('losses', weight_decay)
    return var

In [3]:
def conv3d_layer(tensor_in, w_name, b_name, shape_weight, shape_bias, wd, layer_name='conv', trainable=True):
    with tf.variable_scope(layer_name):
        w = _variable_with_weight_decay(w_name, shape_weight, wd, trainable=trainable)
        b = _variable_on_cpu(b_name, shape_bias, tf.constant_initializer(0.0), trainable=trainable)
        tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, w)
        tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, b)
        
        conv = tf.nn.conv3d(tensor_in, w, strides=[1,1,1,1,1], padding='SAME',name='conv')
        conv_b = tf.nn.bias_add(conv, b, name='conv_bias')
        act = tf.nn.relu(conv_b, 'relu')
        tf.summary.histogram('Convolution_Weights', w)
        tf.summary.histogram('Conv_Biases', b)
        tf.summary.histogram('Conv_Activations', act)
        return act

def fc_layer(tensor_in, w_name, b_name, shape_weight, shape_bias, wd, _dropout, layer_name='fc', trainable=True):
    with tf.variable_scope(layer_name):
        w = _variable_with_weight_decay(w_name, shape_weight, wd, trainable=trainable)
        b = _variable_on_cpu(b_name, shape_bias, tf.constant_initializer(0.0), trainable=trainable)
        #tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, w)
        #tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, b)
        
        tf.add_to_collection('fc_layer', w)
        tf.add_to_collection('fc_layer', b)
        
        fc_1 = tf.matmul(tensor_in, w, name='fc_linear')
        fc_b = tf.nn.bias_add(fc_1, b, name='fc_bias')
        act = tf.nn.relu(fc_b, 'relu')
        tensor_out = tf.nn.dropout(act, _dropout, name='dropout')
        tf.summary.histogram('FC_Weights', w)
        tf.summary.histogram('FC_Biases', b)
        tf.summary.histogram('FC_Activations', act)
        return tensor_out
        
def max_pool_3d(tensor_in, k, layer_name='max_pool'):
    with tf.variable_scope(layer_name):
        k_size=[1,k,2,2,1]
        tensor_out = tf.nn.max_pool3d(tensor_in, ksize=k_size, strides=k_size, padding='SAME')
        return tensor_out
    
def c3d_model(_X, model_settings):
    
    wd = model_settings['weight_decay']
    #Convolution 1a
    conv1 = conv3d_layer(_X, 'wc1a', 'bc1a', [3, 3, 3, 3, 64], [64], None, 'Convolution_1a', False)
    pool1 = max_pool_3d(conv1, 1, 'Max_Pooling_1')
    
    #Convolution 2a
    conv2 = conv3d_layer(pool1, 'wc2a', 'bc2a', [3, 3, 3, 64, 128], [128], None, 'Convolution_2a', False)
    pool2 = max_pool_3d(conv2, 2, 'Max_Pooling_2')
    
    #Convolution 3a, 3b
    conv3a = conv3d_layer(pool2, 'wc3a', 'bc3a',  [3, 3, 3, 128, 256], [256], None, 'Convolution_3a', False)
    conv3b = conv3d_layer(conv3a, 'wc3b', 'bc3b', [3, 3, 3, 256, 256], [256], None, 'Convolution_3b', False)
    pool3 = max_pool_3d(conv3b, 2, 'Max_Pooling_3')
    
    #Convolution 4a, 4b
    conv4a = conv3d_layer(pool3, 'wc4a', 'bc4a',  [3, 3, 3, 256, 512], [512], None, 'Convolution_4a', False)
    conv4b = conv3d_layer(conv4a, 'wc4b', 'bc4b', [3, 3, 3, 512, 512], [512], None, 'Convolution_4b', False)
    pool4 = max_pool_3d(conv4b, 2, 'Max_Pooling_4')
    
    #Convolution 4a, 4b
    conv5a = conv3d_layer(pool4, 'wc5a', 'bc5a',  [3, 3, 3, 512, 512], [512], None, 'Convolution_5a', False)
    conv5b = conv3d_layer(conv5a, 'wc5b', 'bc5b', [3, 3, 3, 512, 512], [512], None, 'Convolution_5b', False)
    pool5 = max_pool_3d(conv5b, 2, 'Max_Pooling_5')

    #Reshape weights for fc6
    pool5 = tf.transpose(pool5, perm=[0,1,4,2,3])
    dense1 = tf.reshape(pool5, [model_settings['batch_size'], 8192])

    #FC6 and FC7
    dense1 = fc_layer(dense1, 'wd1', 'bd1', [8192, 4096], [4096], wd,
                      model_settings['dropout_placeholder'], 'FC6', True)
    dense2 = fc_layer(dense1, 'wd2', 'bd2', [4096, 4096], [4096], wd,
                      model_settings['dropout_placeholder'], 'FC7', True)
    
    model_settings['clip_features'] = dense2
    
    with tf.variable_scope('Softmax_Linear'):
        w_out = _variable_with_weight_decay('wout', [4096, 101], wd, True)
        b_out = _variable_on_cpu('bout', [101], tf.constant_initializer(0.0), True)
        #tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, w_out)
        #tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, b_out)
        out = tf.matmul(dense2, w_out) + b_out
    
    return out

In [4]:
def calc_tower_loss(loss_var_scope, logit, labels):
    with tf.variable_scope(loss_var_scope):
        
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, 
                                                                       logits=logit,
                                                                       name='cross_entropy_per_class')
        cross_entropy_mean = tf.reduce_mean(cross_entropy, name='Cross_Entropy_Mean')
        
        weight_losses = tf.get_collection('losses')
        tf.summary.scalar('Weight_decay_loss', tf.reduce_mean(weight_losses)) 
        
        tf.add_to_collection('losses', cross_entropy_mean)
        total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')
        
        return total_loss

#Updates:
##Add top-n correct predictions for ground up training
def calc_tower_accuracy(logit, labels):
    
    correct_predictions = tf.equal(labels, tf.argmax(logit, 1))
    total_correct = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return total_correct
    
def average_gradients(tower_grads):
    
    average_grads=[]
    for grad_and_vars in zip(*tower_grads):
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

            # Average over the 'tower' dimension.
            grad = tf.concat(axis=0, values=grads)
            grad = tf.reduce_mean(grad, 0)

            # Keep in mind that the Variables are redundant because they are shared
            # across towers. So .. we will just return the first tower's pointer to
            # the Variable.
            v = grad_and_vars[0][1]
            grad_and_var = (grad, v)
            average_grads.append(grad_and_var)
    return average_grads
        
def show_running_info(model_settings, duration, step, loss, accuracy):
    batch_size, num_gpu = model_settings['batch_size'], model_settings['num_gpu']
    num_examples_per_step = model_settings['total_batch']
    examples_per_sec = num_examples_per_step / duration
    sec_per_batch = duration / num_gpu
    format_str = ('%s: step %d, (%.1f examples/sec; %.3f'
                  'sec/batch), (accuracy: %f loss: %f)')
    format_tuple = (datetime.now(), step,
                    examples_per_sec, sec_per_batch, accuracy, loss)
    print (format_str % format_tuple)

In [5]:
def get_UCF101_dir(filename):
    dir_images = []
    start_clip = []
    label_images = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            dir_images.append(row[0])
            start_clip.append(int(row[1]))
            label_images.append(row[2])
    return dir_images, start_clip, label_images

def shuffle_list(dir_clips, start_clips, label_clips, seed=time.time()):
    video_indices = list(range(len(dir_clips)))
    random.shuffle(video_indices)
    shuffled_clip_dirs   = [dir_clips[i] for i in video_indices]
    shuffled_starts = [start_clips[i] for i in video_indices]
    shuffled_labels = [label_clips[i] for i in video_indices]
    return shuffled_clip_dirs, shuffled_starts, shuffled_labels

def get_frames_data(filename, start_index, num_frames_per_clip=16):
    ret_arr = []
    for parent, dirnames, filenames in os.walk(filename):
        if(len(filenames) < num_frames_per_clip):
            return np.array([])
        filenames = sorted(filenames)
        for i in range(start_index - 1, start_index + num_frames_per_clip - 1):
            image_name = str(filename) + '/' + str(filenames[i])
            img = Image.open(image_name)
            img_data = np.array(img)
            ret_arr.append(img_data)
    return ret_arr

def read_clip(dirname, start_index, model_settings):
    num_frames_per_clip = model_settings['frames_per_clip']
    crop_size = model_settings['crop_size']
    np_mean = model_settings['np_mean']
    tmp_data = get_frames_data(dirname, start_index)
    if(len(tmp_data) == 0):
        return np.array([])
    img_datas=[]
    horizontal_flip = random.random()
    for j in range(len(tmp_data)):
        img = Image.fromarray(tmp_data[j].astype(np.uint8))
        if(img.width>img.height):
            scale = float(crop_size)/float(img.height)
            img = np.array(cv2.resize(np.array(img),(int(img.width * scale + 1), crop_size))).astype(np.float32)
        else:
            scale = float(crop_size)/float(img.width)
            img = np.array(cv2.resize(np.array(img),(crop_size, int(img.height * scale + 1)))).astype(np.float32)
        crop_x = int((img.shape[0] - crop_size)/2)
        crop_y = int((img.shape[1] - crop_size)/2)
        img = img[crop_x:crop_x+crop_size, crop_y:crop_y+crop_size,:] - np_mean[j]
        
        #Flip the image 0.5 chance
        if(horizontal_flip > 0.5):
            img = np.fliplr(img)
            
        img_datas.append(img)
    return np.array(img_datas).astype(np.float32)
    

In [6]:
def set_placeholders(model_settings):
    total_batch = model_settings['total_batch']
    images_placeholder = tf.placeholder(tf.float32, shape=model_settings['input_shape'], name="input_clip")
    labels_placeholder = tf.placeholder(tf.int64, shape=(), name="labels")
    dropout_placeholder = tf.placeholder_with_default(model_settings['dropout'], shape=())
    #tf.summary.image('input_batch_sample_image', images_placeholder[:,0], total_batch)
    
    model_settings['images_placeholder'] = images_placeholder
    model_settings['labels_placeholder'] = labels_placeholder
    model_settings['dropout_placeholder'] = dropout_placeholder
    return

def set_queue(model_settings):
    images_placeholder = model_settings['images_placeholder']
    labels_placeholder = model_settings['labels_placeholder']
    
    queue = tf.FIFOQueue(model_settings['queue_size'], 
                         [tf.float32, tf.int64], 
                         shapes=[model_settings['input_shape'],
                                 labels_placeholder.shape],
                        name='Input_Queue')

    enqueue_op = queue.enqueue([model_settings['images_placeholder'], 
                                model_settings['labels_placeholder']], 
                               name='Enqueue_Operation')
    
    model_settings['queue'], model_settings['enqueue_op'] = queue, enqueue_op
    
#Read single clip at a time 
def load_and_enqueue(sess, model_settings, thread_index):
    coord, enqueue_op = model_settings['coord'], model_settings['enqueue_op']
    dir_clips, start_clips, label_clips = model_settings['train_list']
    num_thread = model_settings['num_thread']
    images_placeholder = model_settings['images_placeholder']
    labels_placeholder = model_settings['labels_placeholder']
    data_size = len(dir_clips)
    
    read_index = thread_index
    while not coord.should_stop():        
        clip_dir, start_index, label = dir_clips[read_index], start_clips[read_index], label_clips[read_index]
        clip_dir =  model_settings['data_home'] + clip_dir
        input_clip = read_clip(clip_dir, start_index, model_settings)
        
        if(input_clip.shape == model_settings['input_shape']):
            sess.run(enqueue_op, feed_dict={images_placeholder:input_clip,
                                            labels_placeholder:label})
        read_index = (read_index + num_thread) % data_size
    #print('Stop_requested: %d' % thread_index)
    
def start_queue_threads(sess, model_settings):
    t = []
    for i in range(model_settings['num_thread']):
        t.append(threading.Thread(target=load_and_enqueue, args=(sess, model_settings, i)))
        t[i].start()
    return t

def stop_threads(sess, threads, model_settings):
    coord = model_settings['coord']
    model_settings['coord'].request_stop()
    queue = model_settings['queue']
    for i in range(model_settings['num_thread']):
        if(sess.run(queue.size()) < model_settings['total_batch']):
            break
        sess.run(queue.dequeue_many(model_settings['batch_size']))
    model_settings['coord'].join(threads)
    print('Finished')

In [7]:
def create_forward_graph(model_settings):
    
    queue = model_settings['queue']
    tower_accuracy = []
    tower_loss = []
    with tf.variable_scope(tf.get_variable_scope()):
        for gpu_index in range(model_settings['num_gpu']):
            with tf.device('/gpu:%d' % gpu_index):
                with tf.name_scope('Tower_%d' % gpu_index) as scope:
                    ind_begin, ind_end= (gpu_index * model_settings['batch_size'],
                                         (gpu_index+1) * model_settings['batch_size'])
                    
                    feed_input, feed_label = queue.dequeue_many(model_settings['batch_size'])
                    model_out = c3d_model(feed_input, model_settings)

                    loss = calc_tower_loss(scope, model_out, feed_label)
                    accuracy = calc_tower_accuracy(model_out, feed_label)
                    tower_loss.append(loss)
                    tower_accuracy.append(accuracy)

                    tf.get_variable_scope().reuse_variables()
                    
    tower_mean_loss = tf.reduce_mean(tower_loss)
    tower_mean_accuracy = tf.reduce_mean(tower_accuracy)
    tf.summary.scalar('Total_Loss', tower_mean_loss)
    tf.summary.scalar('Top1_Correct_Predictions', tower_mean_accuracy)
    
    model_settings['tower_mean_loss'] = tower_mean_loss
    model_settings['tower_mean_accuracy'] = tower_mean_accuracy
    model_settings['tower_loss'] = tower_loss
                    
def create_backward_graph(model_settings):
    
    global_step=tf.get_variable('Global_Step',[],
                                initializer=tf.constant_initializer(0),
                                trainable=False)
    
    opt, tower_loss = model_settings['optimizer'], model_settings['tower_loss']
    
    tower_grads=[]
    with tf.variable_scope(tf.get_variable_scope()):
        for gpu_index in range(model_settings['num_gpu']):
            with tf.device('/gpu:%d' % gpu_index):
                with tf.name_scope('Tower_%d' % gpu_index) as scope:
                    loss = tower_loss[gpu_index]
                    grads = opt.compute_gradients(loss)
                    tower_grads.append(grads)
    grads = average_gradients(tower_grads)
    
    apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)
    variable_averages = tf.train.ExponentialMovingAverage(model_settings['moving_decay'], global_step)
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    train_op = tf.group(apply_gradient_op, variable_averages_op)

    summary_op = tf.summary.merge_all()
    
    model_settings['train_op'], model_settings['summary_op'] = train_op, summary_op


def run_training(sess, model_settings):
    
    model_settings['optimizer'] = tf.train.AdamOptimizer(model_settings['learning_rate'])

    #Set placeholders, queue operations and thread coordinator
    set_placeholders(model_settings)
    set_queue(model_settings)
    
    #Create Graph
    create_forward_graph(model_settings)
    create_backward_graph(model_settings)
    
    if(model_settings['fc_pretrained'] == True):
        for fc_variable in tf.get_collection('fc_layer'):
            tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, fc_variable)
    
    #Read training file locations and shuffle
    dir_clips, start_clips, label_clips = get_UCF101_dir(model_settings['train_file_loc'])
    model_settings['train_list'] = shuffle_list(dir_clips, start_clips, label_clips)

    #Initialize file thread Coordinator and Start input reading threads
    model_settings['coord'] = tf.train.Coordinator()
    read_threads = start_queue_threads(sess, model_settings)

    #Save Only Model Variables
    saver = tf.train.Saver(tf.model_variables())
    model_settings['saver'] = saver
    
    #Initialize variables
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    #Restore saved model variables
    if(model_settings['read_pretrained_model'] == True):
        saver.restore(sess, model_settings['model_read_dir'])
        
    #Tensorboard summary writers
    summary_writer = tf.summary.FileWriter(model_settings['checkpoint_dir'])
    summary_writer.add_graph(sess.graph)
    model_settings['summary_writer'] = summary_writer
    
    train_op, summary_op = model_settings['train_op'], model_settings['summary_op']
    
    #Single tower loss implementation for now
    loss_op, acc_op = model_settings['tower_mean_loss'], model_settings['tower_mean_accuracy']
    
    #time.sleep(15)
    
    print('Training begins:')
    for step in range(model_settings['max_steps']):
        start_time = time.time()        
        #print(sess.run(model_settings['queue'].size()))
        
        if (step+1) % model_settings['checkpoints'] != 0:
            _, tower_mean_loss, tower_mean_acc = sess.run([train_op, loss_op, acc_op])
        else:
            _, summary_str, tower_mean_loss, tower_mean_acc = sess.run([train_op, summary_op, loss_op, acc_op])
            summary_writer.add_summary(summary_str, step)
        
        show_running_info(model_settings, time.time() - start_time,
                          step, tower_mean_loss, tower_mean_acc)
    stop_threads(sess, read_threads, model_settings)

def run_testing(sess, model_settings):
    
    #Convert to multi-tower setting
    print('Testing begins:')
    loss_op, acc_op = model_settings['tower_loss'][0], model_settings['tower_accuracy'][0]
    np_mean = np.load('crop_mean.npy').reshape([model_settings['frames_per_clip'],
                                                model_settings['crop_size'],
                                                model_settings['crop_size'],
                                                model_settings['channels']])
    
    data_index = 0
    epoch_finished = False
    batch_accuracy = []
    batch_loss = []
    while(not epoch_finished):
        start_time = time.time()
        
        test_images, test_labels, data_index, epoch_finished = read_clip_label_sequentially(
            dir_clips, label_clips, np_mean, data_index,
            batch_size=model_settings['batch_size'] * model_settings['num_gpu'],
            num_frames_per_clip=model_settings['frames_per_clip'],
            crop_size= model_settings['crop_size'])
        
        img_process_duration = time.time() - start_time
        
        feed_dict = {model_settings['images_placeholder'] : test_images,
                    model_settings['labels_placeholder'] : test_labels,
                    model_settings['dropout_placeholder'] : 1.0}
        
        accuracy, loss = sess.run([acc_op, loss_op], feed_dict=feed_dict)
        
        batch_accuracy.append(accuracy)
        batch_loss.append(loss)
        print('Data index :%d' % data_index)
    
    test_accuracy = np.mean(np.array(batch_accuracy))
    test_loss = np.mean(np.array(batch_loss))
    print('Test accuracy %f, test loss %f!' % (test_accuracy, test_loss))
    
def calculate_features(sess, model_settings):
#Convert to multi-tower setting
    feature_extract_op = model_settings['clip_features']
    np_mean = np.load('crop_mean.npy').reshape([model_settings['frames_per_clip'],
                                                model_settings['crop_size'],
                                                model_settings['crop_size'],
                                                model_settings['channels']])
    
    dir_clips, label_clips = get_UCF101_dir(model_settings['feature_file_loc'])
    dir_clips, label_clips = shuffle_list(dir_clips, label_clips)
    
    data_index = 0
    epoch_finished = False
    x_data = []
    y_data = []
    data_scanned = 0
    while(not epoch_finished):
        #start_time = time.time()
        
        test_images, test_labels, data_index, epoch_finished = read_clip_label_sequentially(
            dir_clips, label_clips, np_mean, data_index,
            batch_size=model_settings['batch_size'] * model_settings['num_gpu'],
            num_frames_per_clip=model_settings['frames_per_clip'],
            crop_size= model_settings['crop_size'])
        data_scanned = data_scanned + model_settings['batch_size']
        #img_process_duration = time.time() - start_time
        
        feed_dict = {model_settings['images_placeholder'] : test_images,
                    model_settings['labels_placeholder'] : test_labels,
                    model_settings['dropout_placeholder'] : 1.0}
        
        x_data.append(sess.run(feature_extract_op, feed_dict=feed_dict))
        y_data.append(test_labels)
        
        print('Data index :%d' % data_index)
    X_data = np.array(x_data)
    Y_data = np.array(y_data)
    X_data = X_data.reshape([data_scanned, 4096])
    Y_data = Y_data.reshape([data_scanned])
    return X_data, Y_data
    
def save_graph(sess, model_settings):
    model_settings['saver'].save(sess, 
                             model_settings['model_save_dir'],
                             write_meta_graph=False)

In [8]:
model_settings = {'max_steps' : 10000, 'batch_size' : 25, 'frames_per_clip' : 16, 
                  'crop_size' : 112, 'num_gpu' : 1, 'channels' : 3,
                  'checkpoint_dir' : './checkpoints/allrand_fc_train',
                  'model_read_dir' : './models/s1m_mod.model',
                  'model_save_dir' : './models/C3D_1.model',
                  'moving_decay' : 0.9999, 'weight_decay' : 0.00005, 'dropout' : 0.5,
                  'learning_rate' : 1e-4, 'checkpoints' : 100,
                  'train_file_loc' : './data/dir_files/train_ucf.csv',
                  'test_file_loc' : './data/dir_files/test_ucf.csv', 
                  'data_home' : './data/',
                  'feature_file_loc' : './data/dir_files/test_ucf_full.lst',
                  'num_thread' : 8, 'queue_size' : 1500,
                  'read_pretrained_model' : True}

model_settings['total_batch'] = model_settings['batch_size'] * model_settings['num_gpu']
model_settings['input_shape'] = (model_settings['frames_per_clip'],
                                 model_settings['crop_size'],
                                 model_settings['crop_size'],
                                 model_settings['channels'])
model_settings['np_mean'] = np.load('crop_mean.npy').reshape(model_settings['input_shape'])

In [ ]:
model_settings['fc_pretrained'] = True
tf.reset_default_graph()
with tf.device('/cpu:0'):
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True,log_device_placement=False)) as sess:
        run_training(sess, model_settings)
        save_graph(sess, model_settings)

print('\n-------------------------------------\nTraining 2!\n')
model_settings['fc_pretrained'] = False
model_settings['checkpoint_dir'] = './checkpoints/lastrand_fc_train'
model_settings['model_save_dir'] = './models/C3D_2.model'
model_settings['max_steps'] = 15000
tf.reset_default_graph()
with tf.device('/cpu:0'):
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True,log_device_placement=False)) as sess:
        run_training(sess, model_settings)
        save_graph(sess, model_settings)

INFO:tensorflow:Restoring parameters from ./models/s1m_mod.model


INFO:tensorflow:Restoring parameters from ./models/s1m_mod.model


Training begins:
2018-01-22 18:20:21.247922: step 0, (8.8 examples/sec; 2.845sec/batch), (accuracy: 0.040000 loss: 5.854966)
2018-01-22 18:20:21.761287: step 1, (48.8 examples/sec; 0.512sec/batch), (accuracy: 0.080000 loss: 5.879155)
2018-01-22 18:20:22.259613: step 2, (50.4 examples/sec; 0.496sec/batch), (accuracy: 0.000000 loss: 5.900627)
2018-01-22 18:20:22.757937: step 3, (50.6 examples/sec; 0.494sec/batch), (accuracy: 0.000000 loss: 5.782173)
2018-01-22 18:20:23.271303: step 4, (48.8 examples/sec; 0.512sec/batch), (accuracy: 0.000000 loss: 5.284870)
2018-01-22 18:20:23.788679: step 5, (48.7 examples/sec; 0.513sec/batch), (accuracy: 0.040000 loss: 5.010622)
2018-01-22 18:20:24.293019: step 6, (49.8 examples/sec; 0.502sec/batch), (accuracy: 0.000000 loss: 5.023072)
2018-01-22 18:20:24.788338: step 7, (50.6 examples/sec; 0.494sec/batch), (accuracy: 0.040000 loss: 4.594900)
2018-01-22 18:20:25.277637: step 8, (51.2 examples/sec; 0.488sec/batch), (accuracy: 0.160000 loss: 4.402936)
201

In [ ]:
#Add last parameters for saving
print('trainable_variables:')
for i in tf.trainable_variables():
    print(i)

print('Model_variables before:')
for i in tf.model_variables():
    print(i)

print('Model_var added variables')
for i in tf.trainable_variables()[4:6]:
    print(i)
    tf.add_to_collection(tf.GraphKeys.MODEL_VARIABLES, i)
    
print('Model_variables after:')
for i in tf.model_variables():
    print(i)

In [ ]:
#Save all parameters
saver_full = tf.train.Saver(tf.model_variables())
model_settings['saver'] = saver_full
save_graph(sess, model_settings)

In [ ]:
#Calculate new data features
sess.close()
model_settings['feature_file_loc'] = './data/dir_files/data_fight1000.list'
tf.reset_default_graph()
with tf.device('/cpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,log_device_placement=False))
    create_forward_graph(model_settings)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    saver = tf.train.Saver(tf.model_variables())
    saver.restore(sess, './models/UCF_finetuneFC_last.model')
    X_data, y_data = calculate_features(sess, model_settings)
    X_data.tofile('./data/features/s1m_f1000_x.npy')
    y_data.tofile('./data/features/s1m_f1000_y.npy')

In [ ]:
X_data.tofile('./data/features/s1m_ucf10_3_x.npy')
y_data.tofile('./data/features/s1m_ucf10_3_y.npy')